In [ ]:
from argparse import ArgumentParser
from operator import gt, index
from typing import Dict, List, Optional, Set, Tuple, Union
import open3d as o3d
import numpy as np
import sys
import matplotlib.pyplot as plt 
import os
from tqdm.notebook import tqdm

In [ ]:
class Plane:
    def __init__(self, points: np.ndarray, index: int, name: str)-> None:
        self.inliers = points
        self.index = index
        self.name = name
        self.voxels = set()
        self.correspondence: float = -1  # -1: not found
        self.found = False
        self.gt_index: Union[int, None] = None

In [ ]:
def read_planes(folder: str)-> List[Plane]:
    planes: List[Plane] = []
    for filename in tqdm(os.listdir(folder)):
        if filename.endswith('.txt'):
            # only use x,y,z values (ignore rgb)
            points = np.loadtxt(f'{folder}/{filename}', usecols=(0,1,2))
            name = filename.replace('.txt', '')
            planes.append(Plane(points, len(planes), name))
        else:
            points = []
            with open(f'{folder}/{filename}', 'r') as file:
                for line in file.readlines()[12:]:
                    l = line.split(' ')[:3]
                    points.append([float(v) for v in l])
            points = np.array(points)
            name = filename.replace('.pcd', '')
            planes.append(Plane(points, len(planes), name))
    return planes

In [130]:
def calc_plane_voxels(planes: List[Plane], voxel_grid: o3d.geometry.VoxelGrid)-> None:
    # calculate corresponding voxels for each plane
    for plane in planes:
        for point in plane.inliers:
            corr_voxel = tuple(voxel_grid.get_voxel(point))
            plane.voxels.add(corr_voxel)

In [201]:
def get_correspondence(ground_truth: List[Plane], a_plane: Plane):
    inlier_count: Dict[int, int] = dict()
    inlier_count[-1] = 0
    for a_vox in a_plane.voxels:
        found = False
        for gt_plane in ground_truth:
            if a_vox in gt_plane.voxels:
                if gt_plane.index not in inlier_count.keys():
                    inlier_count[gt_plane.index] = 0
                inlier_count[gt_plane.index] += 1
                found = True
                break
    best = -1
    for index, count in inlier_count.items():
        if count > inlier_count[best]:
            best = index
    if best == -1 or inlier_count[best] < len(a_plane.inliers) / 2:
        return None
    ground_truth[best].found = True
    return ground_truth[best]

In [ ]:

def find_correspondences(groundtruth: List[Plane], algo_planes: List[Plane]) -> Dict[Plane, Union[Plane, None]]:
    corr: Dict[Plane, Union[Plane, None]] = dict()
    for a_plane in tqdm(algo_planes):
        corr[a_plane] = get_correspondence(groundtruth, a_plane)
        print(corr[a_plane])
    return corr


In [143]:
def get_precision(correspondences: Dict[Plane, Union[Plane, None]], grid: o3d.geometry.VoxelGrid)-> float:
    all_leaves = set()
    correct_leaves = set()
    for a_plane, gt_plane in tqdm(correspondences.items()):
        tmp_all = 0
        tmp_c = 0
        for a_in in a_plane.inliers:
            voxel = tuple(grid.get_voxel(a_in))
            all_leaves.add(voxel)
            tmp_all += 1
            if gt_plane != None and a_in in  gt_plane.inliers:
                tmp_c += 1
                correct_leaves.add(voxel)
        print(f'{tmp_c / tmp_all = }')
    return len(correct_leaves) / len(all_leaves)

In [198]:
def get_recall(ground_truth: List[Plane],correspondences: Dict[Plane, Union[Plane, None]], grid: o3d.geometry.VoxelGrid) -> float:
    all_leaves = set()
    correct_leaves = set()
    for g_plane in tqdm(ground_truth):
        for g_in in g_plane.inliers:
            voxel = tuple(grid.get_voxel(g_in))
        # for voxel in g_plane.voxels:
            all_leaves.add(voxel)
            for a_plane, gt_plane in correspondences.items():
                if gt_plane == g_plane and g_in in a_plane.inliers:
                    correct_leaves.add(voxel)
                    break
    return len(correct_leaves) / len(all_leaves)



### MAIN

In [174]:
# setup
data = [
    "./Stanford3dDataset_v1.2_Aligned_Version/Area_4/conferenceRoom_2/conferenceRoom_2.txt_out.pcd",
    "Stanford3dDataset_v1.2_Aligned_Version/Area_4/conferenceRoom_2/testOPS",
    "Stanford3dDataset_v1.2_Aligned_Version/Area_4/conferenceRoom_2/GT",
    "Stanford3dDataset_v1.2_Aligned_Version/Area_3/WC_1/WC_1.pcd",
    "Stanford3dDataset_v1.2_Aligned_Version/Area_3/WC_1/OPS",
    "Stanford3dDataset_v1.2_Aligned_Version/Area_3/WC_1/WC_1_GT/PCD",
    ]

pcd_file = data[0]
planes_folder = data[1]
gt_folder = data[2]
# pcd_file = "Stanford3dDataset_v1.2_Aligned_Version/Area_4/conferenceRoom_2/conferenceRoom_2.txt_out.pcd"
# planes_folder = "Stanford3dDataset_v1.2_Aligned_Version/Area_4/conferenceRoom_2/OPS"
# gt_folder = "Stanford3dDataset_v1.2_Aligned_Version/Area_4/conferenceRoom_2/GT"


In [175]:
pcd = o3d.io.read_point_cloud(pcd_file)
grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd,voxel_size=0.05)
o3d.visualization.draw_geometries([grid])

In [177]:
points = np.array([p.inliers for p in ground_truth])
gt_pcd = o3d.geometry.PointCloud()
gt_pcd.points = o3d.utility.Vector3dVector(points[0])
gt_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(gt_pcd,voxel_size=0.05)
o3d.visualization.draw_geometries([gt_grid])

/tmp/ipykernel_289475/2843875347.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  points = np.array([p.inliers for p in ground_truth])


In [176]:
ground_truth = read_planes(gt_folder)
algo_planes = read_planes(planes_folder)

  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

In [178]:
calc_plane_voxels(ground_truth,grid)
calc_plane_voxels(algo_planes,grid)

In [202]:
correspondences = find_correspondences(ground_truth,algo_planes)

  0%|          | 0/8 [00:00<?, ?it/s]

In [180]:
precision = get_precision(correspondences, grid)
print(f'{precision = }')


  0%|          | 0/8 [00:00<?, ?it/s]

tmp_c / tmp_all = 0.9992743105950653
tmp_c / tmp_all = 1.0
tmp_c / tmp_all = 1.0
tmp_c / tmp_all = 1.0
tmp_c / tmp_all = 1.0
tmp_c / tmp_all = 1.0
tmp_c / tmp_all = 1.0
tmp_c / tmp_all = 1.0
precision = 0.999940716148921


In [199]:

recall = get_recall(ground_truth, correspondences, grid)
print(f'{recall = }')

  0%|          | 0/45 [00:00<?, ?it/s]

recall = 0.5369752784236935


In [182]:

f1 = (precision * recall) / (precision + recall)
print(f'{f1 = }')

f1 = 0.3493642114190942


In [222]:
found_pts = np.array([p.inliers for p in ground_truth if p.found])
P = []
for pts in found_pts:
    for ps in pts: 
        P.append(ps)
P = np.array(P)
gt_pcd = o3d.geometry.PointCloud()
gt_pcd.points = o3d.utility.Vector3dVector(P)
gt_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(gt_pcd,voxel_size=0.05)
o3d.visualization.draw_geometries([gt_grid])

/tmp/ipykernel_289475/3189615615.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  found_pts = np.array([p.inliers for p in ground_truth if p.found])


RuntimeError: Unable to cast Python instance to C++ type (compile in debug mode for details)